# Importing Libraries and Loading data

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
# from google.colab import drive
import pandas as pd
import re
import spacy
from tqdm import tqdm
import time
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
    confusion_matrix,
    classification_report
)
from tqdm import tqdm
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import GridSearchCV
import pickle

# drive.mount('/content/drive')
nlp = spacy.load('en_core_web_sm')

In [2]:
train = pd.read_csv("Data/original_data/train.csv", low_memory=False)
print(train.shape)

test = pd.read_csv("Data/original_data/test.csv", low_memory=False)
print(test.shape)

(93686, 3)
(31229, 3)


## Data Processing

### Duplicates removal

In [ ]:
train = train.drop_duplicates('crimeaditionalinfo')
print(train.shape)

train = train[~train['crimeaditionalinfo'].isnull()].reset_index(drop=True)
print(train.shape)

test = test[~test['crimeaditionalinfo'].isnull()].reset_index(drop=True)
print(test.shape)

(85014, 3)
(85013, 3)
(31222, 3)


## Data Pipelines

### SLL, SRLL, SRULL

In [ ]:
def do_correction(text):
    '''
    Correct misspelled words or hinglish words to English words
    '''
    changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
    'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
    'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
    'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
    'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
    'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
    'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
    'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
    'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
    'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
    'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
    'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
    'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
    'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
    'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
    'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
    'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
    'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
    'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
    'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
    'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
    'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
    'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
    'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
    'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
    'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
    'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
    'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
    'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
    'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
    'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
    'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount', 'intrnet': "internet", "secreate": "secret", "victime": "victim",
    "apka": "your", "apke": "your", 'aapka': 'yours', 'abusive': 'abuse', 'acc': 'account', 'accidentally': 'accident', 'ad': 'advertisement', 'adhar': 'aadhaar', 'alag': 'apart',
    'amt': 'amount', 'apka': 'your', 'apke': 'your', 'app': 'application', 'approval': 'approve', 'attachment': 'attach', 'baar': 'times', 'band': 'close',
    'bankar': 'become', 'banking': 'bank', 'bar': 'times', 'bata': 'tell', 'blackmailing': 'blackmail', 'booking': 'book', 'careful': 'care',
    'chala': 'walked', 'chesaru': 'done', 'clothe': 'cloth', 'complainant': 'complaint', 'completely': 'complete', 'confirmation': 'confirm',
    'constantly': 'constant', 'continuously': 'continuous', 'dal': 'did', 'deduction': 'deduct', 'delivery': 'deliver', 'denge': 'give',
    'dhani': 'rich', 'di': 'gave', 'dijiye': 'give', 'directly': 'direct', 'dist': 'district', 'dunga': 'give', 'firstly': 'first', 'fraudster': 'fraud',
    'fraudulent': 'fraud', 'frezzing': 'freeze', 'froder': 'fraud', 'frustrated': 'frustrate', 'galat': 'wrong', 'gandi': 'dirty', 'getting': 'get',
    'googlepay': 'gpay', 'harassment': 'harass', 'harrasse': 'harass', 'holder': 'hold', 'hone': 'to', 'honestly': 'honest', 'hum': 'we',
    'immediately': 'immediate', 'impersonation': 'impersonate', 'informed': 'inform', 'initially': 'initial', 'insta': 'instagram', 'instal': 'install',
    'instruction': 'instruct', 'investigation': 'investigate', 'isne': 'has', 'ja': 'go', 'jankar': 'knowing', 'jankari': 'information',
    'jiske': 'whose', 'kar': 'do', 'karwa': 'did', 'kat': 'cut', 'kra': 'do', 'kro': 'do', 'kya': 'what', 'lac': 'lakh', 'ladki': 'lady', 'lal': 'red',
    'li': 'took', 'log': 'people', 'lone': 'loan', 'mai': 'i', 'mang': 'demand', 'mentally': 'mental', 'mila': 'found', 'min': 'minimum', 'mo': 'mobile',
    'mob': 'mobile', 'msg': 'message', 'msgs': 'message', 'nam': 'name', 'ne': 'near', 'nos': 'number', 'notifiction': 'notification', 'num': 'number',
    'pahle': 'first', 'pareshan': 'worried', 'pe': 'pay', 'pende': 'pending', 'ph': 'phone', 'pic': 'picture', 'pr': 'but', 'pta': 'know', 'raha': 'remained',
    'recieve': 'receive', 'recive': 'receive', 'recording': 'record', 'reference': 'reference', 'refrence': 'reference', 'refunded': 'refund', 'rha': 'remained',
    'rhi': 'doing', 'sare': 'ok', 'saving': 'save', 'scamme': 'scammer', 'smishe': 'funny', 'st': 'street', 'stange': 'strange', 'suchana': 'information',
    'suchna': 'information', 'suchnakrta': 'inform ', 'terrifying': 'terrify', 'texte': 'text', 'thana': 'station', 'threate': 'threat', 'trans': 'transaction',
    'transacation': 'transaction', 'transaction': 'transaction', 'transction': 'transaction', 'transection': 'transaction', 'usme': 'in that', 'usne': 'he',
    'vishe': 'special', 'wale': 'ones', 'wapas': 'back', 'week': 'weak', 'whatsup': 'whatsapp', 'withdrawal': 'withdraw', 'working': 'work', 'ya': 'yes',
    'hun': 'am', 'nahin': 'no', 'agar': 'if', 'tha': 'was', 'aayi': 'came', 'iske': 'it is', 'unka': 'their', 'dete': 'give', 'kre': 'do',
    'donot':'do not', 'lekar': 'taking', 'march': 'mar', 'january': 'jan', 'february': 'feb', 'april': 'apr', 'june': 'jun', 'july': 'jul',
    'august': 'aug','september':'sept', 'october': 'oct','november': 'nov', 'december': 'dec', 'aisa': 'as such'
    }

    # Correct the words
    corrected_msg = ''
    all_keys = changing_words.keys()
    for word in text.split():
        if word in all_keys:
            corrected_msg += changing_words[word]
        else:
            corrected_msg += word
        corrected_msg += " "
    return corrected_msg

def do_sensitive_website_replacement(corrected_msg, website_match_findings):
  '''
  Replacing sensitive websites with the sensitive word that is matched
  '''
  sensitive_keywords = [
        "porn", "xxx", "hentai", "xvideo", "adult", "breast","pussy", "dick", "cock","sex", "erotic", "nude", "camgirl", "cams", "cheat",
        "escort", "fetish", "bdsm", "taboo", "incest", "shemale", "milf", "barelylegal", "amateur",
        "anal", "fetish", "hotgirls", "kinky", "hardcore", "lingerie", "webcam", "strip", "nsfw",
        "onlyfans", "redtube", "youjizz", "spankbang", "tnaflix", "xtube", "nudity", "playboy",
        "bangbros", "brazzers", "naughty", "mature", "teens", "voyeur", "threesome", "gangbang",
        "peeping", "exposed", "babes", "slut", "masturbate", "hookup", "sugarbaby", "paypig", "comic"
  ]

  new_msg = ''
  for word in corrected_msg.strip().split():
    # print(word)
    if len(website_match_findings) > 0:
        if word in website_match_findings and 'express' not in word and 'gmail' not in word and 'internet' not in word:
            # Check if it has sensitive word
            for sen_word in sensitive_keywords:
                if sen_word in word:
                    new_msg += sen_word
        else:
          new_msg += word
    else:
        new_msg += word
    new_msg += " "
  return new_msg

In [ ]:
## DATA PROCESSING
'''
1. stopwords
2. lower
3. replace mispelled words
4. detect websites and detect sensitive words and replace them, else remove
5. lemma
'''
stop_words = [
    "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according",
    "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag",
    "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although",
    "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone",
    "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar",
    "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available",
    "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes",
    "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside",
    "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief",
    "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause",
    "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co",
    "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding",
    "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d",
    "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different",
    "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due",
    "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el",
    "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et",
    "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey",
    "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo",
    "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full",
    "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes",
    "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt",
    "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein",
    "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully",
    "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id",
    "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch",
    "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward",
    "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr",
    "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last",
    "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely",
    "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make",
    "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million",
    "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't",
    "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't",
    "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless",
    "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained",
    "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only",
    "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over",
    "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas",
    "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible",
    "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud",
    "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re",
    "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research",
    "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt",
    "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section",
    "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf",
    "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown",
    "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm",
    "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp",
    "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such",
    "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten",
    "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves",
    "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto",
    "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this",
    "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til",
    "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying",
    "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately",
    "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses",
    "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs",
    "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went",
    "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's",
    "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither",
    "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish",
    "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't",
    "xt", "xv", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself",
    "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz", 'ag', 'amit', 'ani', 'dec', 'delhi', 'devi', 'feb', 'five', 'gupta', 'hi', 'hui',
    'jaipur', 'jan', 'kumar', 'may', 'rahul', 'ram', 'sharma', 'singh', 'yadav', 'mar', 'apr', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov']

train_SRULL = []
train_SLL = []
train_SRLL = []

for msg in tqdm(train['crimeaditionalinfo'].tolist(), desc = 'processing train:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SRULL.append(' '.join(lemmatized_tokens).strip())

train['SRULL_msg'] = train_SRULL
train['SLL_msg'] = train_SLL
train['SRLL_msg'] = train_SRLL

processing train:: 100%|██████████| 85013/85013 [27:33<00:00, 51.40it/s]


Train saved


In [ ]:
test_SLL = []
test_SRLL = []
test_SRULL = []

for msg in tqdm(test['crimeaditionalinfo'], desc = 'processing test:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SRULL.append(' '.join(lemmatized_tokens).strip())

test['SLL_msg'] = test_SLL
test['SRLL_msg'] = test_SRLL
test['SRULL_msg'] = test_SRULL

processing test:: 100%|██████████| 31222/31222 [09:41<00:00, 53.71it/s]


Test saved


In [ ]:
train.to_csv("Data/processed_data/final_train.csv",index=False)
print("Train saved")

test.to_csv("Data/processed_data/final_test.csv",index=False)
print("Test saved")

Train saved
Test saved


## Category grouping

**Label: 0 (Women/Child Related Crime)**
1. Sexually Obscene material
2. Sexually Explicit Act
3. Child Pornography CPChild Sexual Abuse Material CSAM
4. RapeGang Rape RGRSexually Abusive Content

**Label: 1 (Financial Crime)**
1. Online Financial Fraud
2. Cryptocurrency Crime
3. Online Gambling  Betting

**Label: 2 (Other Cyber Crime)**
1. Online and Social Media Related Crime
2. Any Other Cyber Crime
3. Cyber Attack/ Dependent Crimes
4. Hacking  Damage to computercomputer system etc
5. Cyber Terrorism
6. Ransomware
7. Online Cyber Trafficking
8. Report Unlawful Content

In [6]:
train = pd.read_csv("Data/processed_data/final_train.csv", low_memory=False)
print(train.shape)

test = pd.read_csv("Data/processed_data/final_test.csv", low_memory=False)
print(test.shape)

(84040, 13)
(28328, 13)


In [ ]:
def create_final_category(val):
  if val in ['Sexually Obscene material', 'Sexually Explicit Act', 'Child Pornography CPChild Sexual Abuse Material CSAM',
 'RapeGang Rape RGRSexually Abusive Content']:
    return 0
  elif val in ['Online Financial Fraud', 'Cryptocurrency Crime', 'Online Gambling  Betting']:
    return 1
  elif val in ['Online and Social Media Related Crime',
 'Any Other Cyber Crime',
 'Cyber Attack/ Dependent Crimes',
 'Hacking  Damage to computercomputer system etc',
 'Cyber Terrorism',
 'Ransomware', 'Report Unlawful Content', 'Online Cyber Trafficking']:
    return 2
  else:
    return 3

train['final_category'] = train['category'].apply(create_final_category)
test['final_category'] = test['category'].apply(create_final_category)
print(train.shape, test.shape)

train = train[~train['final_category'].eq(3)].reset_index(drop=True)
test = test[~test['final_category'].eq(3)].reset_index(drop=True)
print(train.shape, test.shape)

(85013, 7) (31218, 7)
(85013, 7) (31218, 7)


# Category Classification

We experimented by building **BoW** and **Tf-IDF** embeddings over **SLL, SRLL, SRULL** processed data and trained on different models - **Logistic Regression, Naive Bayes, Random Forest, CatBoost, Gradient Boosting.** We finalised three pipelines which provides better performance and stability over train and OOT. They are:

1. NB-SRULL-TFIDF: We use SRULL processed data with TF-IDF embedding with max_features = 1000 and classify using Naive Bayes.

2. LR-SRLL-TFIDF: We use SRLL processed data with TF-IDF embedding with max_features = 1000 and classify using Logistic Regression.

3. LR-SRULL-TFIDF: We use SRULL processed data with TF-IDF embedding with max_features = 1000 and classify using Logistic Regression.

In [ ]:
def classwise_scores_and_overall(model, X_test_embed, y_test):

  y_prob = model.predict_proba(X_test_embed)
  y_pred = model.predict(X_test_embed)
  y_test_bin = label_binarize(y_test, classes=[0, 1, 2])

  # Calculate per-class metrics
  metrics = []
  for i, class_label in enumerate(model.classes_):
      class_auc = roc_auc_score(y_test_bin[:, i], y_prob[:, i])
      class_accuracy = accuracy_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_f1 = f1_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_recall = recall_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_precision = precision_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_loss = log_loss(y_test_bin[:, i], y_prob[:, i])

      metrics.append({
          'Class': class_label,
          'AUC': class_auc,
          'Accuracy': class_accuracy,
          'F1 Score': class_f1,
          'Recall': class_recall,
          'Precision': class_precision,
          'Log Loss': class_loss
      })

  metrics_df = pd.DataFrame(metrics)

  # Calculate overall metrics
  overall_accuracy = accuracy_score(y_test, y_pred)
  overall_f1 = f1_score(y_test, y_pred, average='weighted')
  overall_recall = recall_score(y_test, y_pred, average='weighted')
  overall_precision = precision_score(y_test, y_pred, average='weighted')
  overall_auc = roc_auc_score(y_test_bin, y_prob, average='weighted', multi_class='ovr')
  overall_loss = log_loss(y_test_bin, y_prob)

  # Display overall metrics
  overall_metrics = {
      'Overall Accuracy': overall_accuracy,
      'Overall F1 Score': overall_f1,
      'Overall Recall': overall_recall,
      'Overall Precision': overall_precision,
      'Overall AUC': overall_auc,
      'Overall Log Loss': overall_loss
  }
  overall_metrics_df = pd.DataFrame([overall_metrics])
  return metrics_df, overall_metrics_df

## NB-SRULL-TFIDF

In [ ]:
print(train.shape, test.shape)
train = train[~train['SRULL_msg'].isnull()].reset_index(drop=True)
test = test[~test['SRULL_msg'].isnull()].reset_index(drop=True)
print(train.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train['SRULL_msg'], train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category'])
print(X_train.shape, X_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
with open("assets/CategoryClassification/NB_SRULL_TFIDF/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
oot_tfidf = tfidf_vectorizer.transform(test['SRULL_msg'])
overall_tfidf = tfidf_vectorizer.transform(train['SRULL_msg'])

y_oot = test['final_category'].tolist()
y_train_overall = train['final_category'].tolist()

(84421, 7) (30791, 7)
(84421, 7) (30791, 7)
(63315,) (21106,)


In [ ]:
# model = MultinomialNB(alpha=5, fit_prior=True)
with open("assets/CategoryClassification/NB_SRULL_TFIDF/naive_bayes.pkl", "rb") as f:
    model = pickle.load(f)

# model.fit(X_train_tfidf, y_train)

train_class_wise, train_overall = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise, test_overall = classwise_scores_and_overall(model,X_test_tfidf, y_test)
oot_class_wise, oot_overall = classwise_scores_and_overall(model,oot_tfidf, y_oot)
train_overall_class_wise, train_overall_overall = classwise_scores_and_overall(model,overall_tfidf, y_train_overall)

classwise = pd.concat([train_class_wise, test_class_wise, train_overall_class_wise, oot_class_wise])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2',
                   'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall, test_overall, train_overall_overall, oot_overall])
overall.index = ['train', 'test', 'train_overall','oot']
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.781600,0.783758,0.782140,0.763567
Overall F1 Score,0.765259,0.767189,0.765749,0.737903
Overall Recall,0.781600,0.783758,0.782140,0.763567
Overall Precision,0.768122,0.769679,0.768533,0.743689
Overall AUC,0.889339,0.889313,0.889335,0.896500
Overall Log Loss,0.484507,0.485958,0.484870,0.491247


## LR-SRLL-TFIDF

In [ ]:
print(train.shape, test.shape)
train = train[~train['SRLL_msg'].isnull()].reset_index(drop=True)
test = test[~test['SRLL_msg'].isnull()].reset_index(drop=True)
print(train.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train['SRLL_msg'], train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category'])
print(X_train.shape, X_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features = 1000)  # You can customize max_features
with open("assets/CategoryClassification/LR_SRLL_TFIDF/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
oot_tfidf = tfidf_vectorizer.transform(test['SRLL_msg'])
overall_tfidf = tfidf_vectorizer.transform(train['SRLL_msg'])

y_oot = test['final_category'].tolist()
y_train_overall = train['final_category'].tolist()

(84421, 7) (30791, 7)
(84421, 7) (30791, 7)
(63315,) (21106,)


In [ ]:
# model = LogisticRegression(C=0.1, class_weight=None, penalty='l2', solver='lbfgs')
with open("assets/CategoryClassification/LR_SRLL_TFIDF/logistic_regression.pkl", "rb") as f:
    model = pickle.load(f)

# model.fit(X_train_tfidf, y_train)


train_class_wise, train_overall = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise, test_overall = classwise_scores_and_overall(model,X_test_tfidf, y_test)
oot_class_wise, oot_overall = classwise_scores_and_overall(model,oot_tfidf, y_oot)
train_overall_class_wise, train_overall_overall = classwise_scores_and_overall(model,overall_tfidf, y_train_overall)

classwise = pd.concat([train_class_wise, test_class_wise, train_overall_class_wise, oot_class_wise])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2',
                   'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall, test_overall, train_overall_overall, oot_overall])
overall.index = ['train', 'test', 'train_overall','oot']
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.799874,0.794608,0.798557,0.778961
Overall F1 Score,0.784507,0.780257,0.783444,0.753659
Overall Recall,0.799874,0.794608,0.798557,0.778961
Overall Precision,0.793094,0.788995,0.792064,0.772064
Overall AUC,0.906029,0.901644,0.904939,0.908307
Overall Log Loss,0.456988,0.465809,0.459193,0.485340


## LR-SRULL-TFIDF

In [ ]:
print(train.shape, test.shape)
train = train[~train['SRULL_msg'].isnull()].reset_index(drop=True)
test = test[~test['SRULL_msg'].isnull()].reset_index(drop=True)
print(train.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train['SRULL_msg'], train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category'])
print(X_train.shape, X_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features = 1000)  # You can customize max_features
with open("assets/CategoryClassification/LR_SRULL_TFIDF/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
oot_tfidf = tfidf_vectorizer.transform(test['SRULL_msg'])
overall_tfidf = tfidf_vectorizer.transform(train['SRULL_msg'])

y_oot = test['final_category'].tolist()
y_train_overall = train['final_category'].tolist()

(84421, 7) (30791, 7)
(84421, 7) (30791, 7)
(63315,) (21106,)


In [ ]:
# 'class_weight': None, 'max_iter': 500, 'solver': 'lbfgs'

# model = LogisticRegression(class_weight = None, max_iter = 500, solver = 'lbfgs')
with open("assets/CategoryClassification/LR_SRULL_TFIDF/logistic_regression.pkl", "rb") as f:
    model = pickle.load(f)
# model.fit(X_train_tfidf, y_train)

train_class_wise, train_overall = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise, test_overall = classwise_scores_and_overall(model,X_test_tfidf, y_test)
oot_class_wise, oot_overall = classwise_scores_and_overall(model,oot_tfidf, y_oot)
train_overall_class_wise, train_overall_overall = classwise_scores_and_overall(model,overall_tfidf, y_train_overall)

classwise = pd.concat([train_class_wise, test_class_wise, train_overall_class_wise, oot_class_wise])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2',
                   'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall, test_overall, train_overall_overall, oot_overall])
overall.index = ['train', 'test', 'train_overall','oot']
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.804738,0.795035,0.802312,0.778442
Overall F1 Score,0.795570,0.786121,0.793204,0.760164
Overall Recall,0.804738,0.795035,0.802312,0.778442
Overall Precision,0.797439,0.787702,0.794987,0.767189
Overall AUC,0.910084,0.901564,0.907952,0.911025
Overall Log Loss,0.436617,0.457220,0.441768,0.457033


# Sub-category Classification

In [2]:
import pandas as pd

train = pd.read_csv("Data/processed_data/final_train.csv",low_memory=False)
print(train.shape)

test = pd.read_csv("Data/processed_data/final_test.csv",low_memory=False)
print(test.shape)

def create_final_category(val):
  if val in ['Sexually Obscene material', 'Sexually Explicit Act', 'Child Pornography CPChild Sexual Abuse Material CSAM',
 'RapeGang Rape RGRSexually Abusive Content']:
    return 0
  elif val in ['Online Financial Fraud', 'Cryptocurrency Crime', 'Online Gambling  Betting']:
    return 1
  elif val in ['Online and Social Media Related Crime',
 'Any Other Cyber Crime',
 'Cyber Attack/ Dependent Crimes',
 'Hacking  Damage to computercomputer system etc',
 'Cyber Terrorism',
 'Ransomware', 'Report Unlawful Content', 'Online Cyber Trafficking']:
    return 2
  else:
    return 3

train['final_category'] = train['category'].apply(create_final_category)
test['final_category'] = test['category'].apply(create_final_category)

print(train['sub_category'].isna().sum())
print(train['sub_category'].isnull().sum())

print(test['sub_category'].isna().sum())
print(test['sub_category'].isnull().sum())

train['final_subcategory'] = train['sub_category'].copy()
train.loc[train['sub_category'].isnull(), 'final_subcategory'] = train[train['sub_category'].isnull()]['category']
print(train['final_subcategory'].isnull().sum(), train.shape)

test['final_subcategory'] = test['sub_category'].copy()
test.loc[test['sub_category'].isnull(), 'final_subcategory'] = test[test['sub_category'].isnull()]['category']
print(test['final_subcategory'].isnull().sum(), test.shape)

train = train[~train['SRLL_msg_updated'].isnull()].reset_index(drop=True)
print(train.shape)
test = test[~test['SRLL_msg_updated'].isnull()].reset_index(drop=True)
print(test.shape)

category_mappings = {'Cyber Bullying  Stalking  Sexting': 'Cyber Bullying/Stalking/Sexting',
 'Fraud CallVishing': 'Fraud Call/Vishing',
 'Online Gambling  Betting': 'Online Gambling/Betting Fraud',
 'Online Job Fraud': 'Online Job Fraud',
 'UPI Related Frauds': 'UPI-Related Frauds',
 'Internet Banking Related Fraud': 'Internet Banking-Related Fraud',
 'RapeGang Rape RGRSexually Abusive Content': 'Rape/Gang Rape-Sexually Abusive Content',
 'Other': 'Any Other Cyber Crime',
 'Profile Hacking Identity Theft': 'Profile Hacking/Identity Theft',
 'DebitCredit Card FraudSim Swap Fraud': 'Debit/Credit Card/Sim Swap Fraud',
 'EWallet Related Fraud': 'E-Wallet Related Frauds',
 'Data Breach/Theft': 'Data Breach/Theft',
 'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks': 'Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks',
 'FakeImpersonating Profile': 'Impersonation Fraud',
 'Cryptocurrency Fraud': 'Cryptocurrency Fraud',
 'Sexually Explicit Act': 'Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Sexually Obscene material': 'Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Malware Attack': 'Malicious code attacks',
 'Business Email CompromiseEmail Takeover': 'Business Email Compromise/Email Takeover',
 'Email Hacking': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Cheating by Impersonation': 'Impersonation Fraud',
 'Hacking/Defacement': 'Hacking/Website Defacement',
 'Unauthorised AccessData Breach': 'Unauthorized Access/Data Breach',
 'SQL Injection': 'Malicious code attacks',
 'Provocative Speech for unlawful acts': 'Provocative Speech of Unlawful Acts',
 'Ransomware Attack': 'Malicious code attacks',
 'Cyber Terrorism': 'Cyber Terrorism',
 'Child Pornography CPChild Sexual Abuse Material CSAM': 'Child Pornography/Child Sexual Abuse Material(CSAM)',
 'Tampering with computer source documents': 'Tampering with Computer Source Documents',
 'DematDepository Fraud': 'Demat/Depository Fraud',
 'Online Trafficking': 'Online Cyber Trafficking',
 'Online Matrimonial Fraud': 'Online Matrimonial Fraud',
 'Website DefacementHacking': 'Hacking/Website Defacement',
 'Damage to computer computer systems etc': 'Damage to Computer Systems',
 'Impersonating Email': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'EMail Phishing': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Ransomware': 'Malicious code attacks',
 'Intimidating Email': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Against Interest of sovereignty or integrity of India': 'Provocative Speech of Unlawful Acts'}

print(test.shape)
test = test[test['final_subcategory'].isin(train['final_subcategory'])].reset_index(drop=True)
print(test.shape)

train['final_subcategory_grouped'] = train['final_subcategory'].replace(category_mappings)
test['final_subcategory_grouped'] = test['final_subcategory'].replace(category_mappings)

(84040, 13)
(28328, 13)
3798
3798
1358
1358
0 (84040, 14)
0 (28328, 14)
(84023, 14)
(28327, 14)
(28327, 14)
(28323, 14)


In [3]:
def post_analysis(df,x_train, x_test, oot, model, overall_feature_bow, x_train_bow, x_test_bow, oot_bow, model_name = 'lr', prediction_feature = 'final_category'):
  from sklearn.metrics import (roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix,classification_report)

  y_test_pred = model.predict(x_test_bow)
  y_test_proba = model.predict_proba(x_test_bow)[:, 1]
  y_train_pred = model.predict(x_train_bow)
  y_train_proba = model.predict_proba(x_train_bow)[:, 1]
  y_overall_pred = model.predict(overall_feature_bow)
  y_overall_proba = model.predict_proba(overall_feature_bow)[:, 1]
  y_oot_pred = model.predict(oot_bow)
  y_oot_proba = model.predict_proba(oot_bow)[:, 1]

  df[model_name+'_model_score'] = model.predict_proba(overall_feature_bow)[:, 1]
  x_train[model_name+'_model_score'] = model.predict_proba(x_train_bow)[:, 1]
  x_test[model_name+'_model_score'] = model.predict_proba(x_test_bow)[:, 1]
  oot[model_name+'_model_score'] = model.predict_proba(oot_bow)[:, 1]

  # Calculate relevant metrics for the training set
  train_metrics = {
      # "AUC": roc_auc_score(y_train, y_train_proba),
      "Accuracy": accuracy_score(y_train, y_train_pred),
      # "Precision": precision_score(y_train, y_test_pred),
      # "Recall": recall_score(y_train, y_test_pred),
      "F1 Score": f1_score(y_train, y_train_pred, average='weighted'),
      # "Log Loss": log_loss(y_train, y_train_proba)
  }

  test_metrics = {
      # "AUC": roc_auc_score(y_test, y_test_proba),
      "Accuracy": accuracy_score(y_test, y_test_pred),
      # "Precision": precision_score(y_test, y_test_pred),
      # "Recall": recall_score(y_test, y_test_pred),
      "F1 Score": f1_score(y_test, y_test_pred, average='weighted'),
      # "Log Loss": log_loss(y_test, y_test_proba)
  }

  overall_metrics = {
      # "AUC": roc_auc_score(df[prediction_feature].tolist(), y_overall_proba),
      "Accuracy": accuracy_score(df[prediction_feature].tolist(), y_overall_pred),
      # "Precision": precision_score(df[prediction_feature].tolist(), y_overall_pred),
      # "Recall": recall_score(df[prediction_feature].tolist(), y_overall_pred),
      "F1 Score": f1_score(df[prediction_feature].tolist(), y_overall_pred, average='weighted'),
      # "Log Loss": log_loss(df[prediction_feature].tolist(), y_overall_proba)
  }

  oot_metrics = {
      # "AUC": roc_auc_score(oot[prediction_feature].tolist(), y_oot_proba),
      "Accuracy": accuracy_score(oot[prediction_feature].tolist(), y_oot_pred),
      # "Precision": precision_score(oot[prediction_feature].tolist(), y_oot_pred),
      # "Recall": recall_score(oot[prediction_feature].tolist(), y_oot_pred),
      "F1 Score": f1_score(oot[prediction_feature].tolist(), y_oot_pred, average='weighted'),
      # "Log Loss": log_loss(oot[prediction_feature].tolist(), y_oot_proba)
  }

  # Create a DataFrame to compare metrics
  metrics_df = pd.DataFrame({
      'Metric': list(train_metrics.keys()),
      'Train': list(train_metrics.values()),
      'Test': list(test_metrics.values()),
      'Overall Data': list(overall_metrics.values()),
      'OOT': list(oot_metrics.values())
  })

  # Display the DataFrame
  return display(metrics_df)

In [4]:
def classwise_scores_and_overall(model, X_test_embed, y_test):

  y_pred = model.predict(X_test_embed)
  # Calculate per-class metrics
  metrics = []
  for i, class_label in enumerate(model.classes_):
      class_accuracy = accuracy_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_f1 = f1_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_recall = recall_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_precision = precision_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))

      metrics.append({
          'Class': class_label,
          'Accuracy': class_accuracy,
          'F1 Score': class_f1,
          'Recall': class_recall,
          'Precision': class_precision
      })

  metrics_df = pd.DataFrame(metrics)

  # Calculate overall metrics
  overall_accuracy = accuracy_score(y_test, y_pred)
  overall_f1 = f1_score(y_test, y_pred, average='weighted')
  overall_recall = recall_score(y_test, y_pred, average='weighted')
  overall_precision = precision_score(y_test, y_pred, average='weighted')

  # Display overall metrics
  overall_metrics = {
      'Overall Accuracy': overall_accuracy,
      'Overall F1 Score': overall_f1,
      'Overall Recall': overall_recall,
      'Overall Precision': overall_precision
  }
  overall_metrics_df = pd.DataFrame([overall_metrics])
  return metrics_df, overall_metrics_df

In [8]:
import pickle

with open("assets/SubCategoryClassification/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [9]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
# label_encoder = LabelEncoder()

with open("assets/SubCategoryClassification/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Fit and transform the subCategory_updated labels
train['subcategory_label'] = label_encoder.transform(train['final_subcategory_grouped'])
test['subcategory_label'] = label_encoder.transform(test['final_subcategory_grouped'])

# Check the mapping of original labels to encoded labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)

Label mapping: {'Any Other Cyber Crime': 0, 'Business Email Compromise/Email Takeover': 1, 'Child Pornography/Child Sexual Abuse Material(CSAM)': 2, 'Cryptocurrency Fraud': 3, 'Cyber Bullying/Stalking/Sexting': 4, 'Cyber Terrorism': 5, 'Damage to Computer Systems': 6, 'Data Breach/Theft': 7, 'Debit/Credit Card/Sim Swap Fraud': 8, 'Demat/Depository Fraud': 9, 'Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks': 10, 'E-Wallet Related Frauds': 11, 'Email Hacking/Phishing/Impersonating/Intimidating': 12, 'Fraud Call/Vishing': 13, 'Hacking/Website Defacement': 14, 'Impersonation Fraud': 15, 'Internet Banking-Related Fraud': 16, 'Malicious code attacks': 17, 'Online Cyber Trafficking': 18, 'Online Gambling/Betting Fraud': 19, 'Online Job Fraud': 20, 'Online Matrimonial Fraud': 21, 'Profile Hacking/Identity Theft': 22, 'Provocative Speech of Unlawful Acts': 23, 'Rape/Gang Rape-Sexually Abusive Content': 24, 'Sale,Publishing and Transmitting Obscene Material/Sexually Exp

In [ ]:
for i in label_mapping.keys():
  print(i)

Any Other Cyber Crime
Business Email Compromise/Email Takeover
Child Pornography/Child Sexual Abuse Material(CSAM)
Cryptocurrency Fraud
Cyber Bullying/Stalking/Sexting
Cyber Terrorism
Damage to Computer Systems
Data Breach/Theft
Debit/Credit Card/Sim Swap Fraud
Demat/Depository Fraud
Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks
E-Wallet Related Frauds
Email Hacking/Phishing/Impersonating/Intimidating
Fraud Call/Vishing
Hacking/Website Defacement
Impersonation Fraud
Internet Banking-Related Fraud
Malicious code attacks
Online Cyber Trafficking
Online Gambling/Betting Fraud
Online Job Fraud
Online Matrimonial Fraud
Profile Hacking/Identity Theft
Provocative Speech of Unlawful Acts
Rape/Gang Rape-Sexually Abusive Content
Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material
Tampering with Computer Source Documents
UPI-Related Frauds
Unauthorized Access/Data Breach


In [ ]:
train['subcategory_label'] = train['subcategory_label'].apply(lambda x: 24 if x==25 else x)
test['subcategory_label'] = test['subcategory_label'].apply(lambda x: 24 if x==25 else x)

category_0_train = train[train['final_category'] == 0].reset_index(drop=True)
category_1_train = train[train['final_category'] == 1].reset_index(drop=True)
category_2_train = train[train['final_category'] == 2].reset_index(drop=True)

category_0_test = test[test['final_category'] == 0].reset_index(drop=True)
category_1_test = test[test['final_category'] == 1].reset_index(drop=True)
category_2_test = test[test['final_category'] == 2].reset_index(drop=True)


In [ ]:
# For category_0_train
category_0_labels = sorted(category_0_train['subcategory_label'].unique())  # Sort the labels in ascending order
category_0_labels_names = category_0_train.groupby('subcategory_label')['final_subcategory_grouped'].first()  # Get the first group name for each label
print("Labels and their corresponding subcategory group in category 0 train:")
for label in category_0_labels:
    print(f"Label {label}: {category_0_labels_names[label]}")

# For category_1_train
category_1_labels = sorted(category_1_train['subcategory_label'].unique())  # Sort the labels in ascending order
category_1_labels_names = category_1_train.groupby('subcategory_label')['final_subcategory_grouped'].first()  # Get the first group name for each label
print("\nLabels and their corresponding subcategory group in category 1 train:")
for label in category_1_labels:
    print(f"Label {label}: {category_1_labels_names[label]}")

# For category_2_train
category_2_labels = sorted(category_2_train['subcategory_label'].unique())  # Sort the labels in ascending order
category_2_labels_names = category_2_train.groupby('subcategory_label')['final_subcategory_grouped'].first()  # Get the first group name for each label
print("\nLabels and their corresponding subcategory group in category 2 train:")
for label in category_2_labels:
    print(f"Label {label}: {category_2_labels_names[label]}")


Labels and their corresponding subcategory group in category 0 train:
Label 2: Child Pornography/Child Sexual Abuse Material(CSAM)
Label 24: Rape/Gang Rape-Sexually Abusive Content

Labels and their corresponding subcategory group in category 1 train:
Label 1: Business Email Compromise/Email Takeover
Label 3: Cryptocurrency Fraud
Label 8: Debit/Credit Card/Sim Swap Fraud
Label 9: Demat/Depository Fraud
Label 11: E-Wallet Related Frauds
Label 13: Fraud Call/Vishing
Label 16: Internet Banking-Related Fraud
Label 19: Online Gambling/Betting Fraud
Label 27: UPI-Related Frauds

Labels and their corresponding subcategory group in category 2 train:
Label 0: Any Other Cyber Crime
Label 4: Cyber Bullying/Stalking/Sexting
Label 5: Cyber Terrorism
Label 6: Damage to Computer Systems
Label 7: Data Breach/Theft
Label 10: Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks
Label 12: Email Hacking/Phishing/Impersonating/Intimidating
Label 14: Hacking/Website Defacement
Label 15: 

##Category_0 - Women/Child	Related	Crime	- Subcategories classification

###Running top 3 models for this category from 5 models(Logistic regression,Naive bayes,random forest,Catboosting,Gradient Boosting)



1.   CatBoosting with TF-IDF
2.   Logistic regression with BOW
3.   Gradient Boosting with TF-IDF



####Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

with open("assets/SubCategoryClassification/category_0/Logistic_Regression/bow_vectorizer.pkl", "rb") as f:
    bow_vectorizer = pickle.load(f)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['subcategory_label'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_0/Logistic_Regression/logistic_regression_model.pkl", "rb") as f:
    logreg = pickle.load(f)

# logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
# logreg.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(logreg, X_train_bow, y_train )
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(logreg, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall( logreg,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(logreg, X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall', 'oot']
overall.T


(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)
2319    24
3409    24
742     24
2093    24
1115    24
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


,train,test,train_overall,oot
Overall Accuracy,0.960647,0.907271,0.947299,0.921887
Overall F1 Score,0.956903,0.895505,0.941883,0.912725
Overall Recall,0.960647,0.907271,0.947299,0.921887
Overall Precision,0.959819,0.890056,0.943378,0.908930


In [ ]:
# import os
# import joblib
# import numpy as np

# # Base directory path
# base_path = "/assets/"
# model_dir = os.path.join(base_path, "SubCategoryClassification/category_0/Logistic_Regression/")

# # Create the folder if it doesn't exist
# os.makedirs(model_dir, exist_ok=True)

# # Define paths for saving
# model_path = os.path.join(model_dir, "logistic_regression_model.pkl")
# bow_vectorizer_path = os.path.join(model_dir, "bow_vectorizer.pkl")

# # Save the trained Logistic Regression model
# joblib.dump(logreg, model_path)
# print(f"Logistic Regression model saved to {model_path}")

# # Save the BOW vectorizer
# joblib.dump(bow_vectorizer, bow_vectorizer_path)
# print(f"TF-IDF vectorizer saved to {bow_vectorizer_path}")


Logistic Regression model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_0/Logistic_Regression/logistic_regression_model.pkl
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_0/Logistic_Regression/bow_vectorizer.pkl


#### CatBoost

In [ ]:
from catboost import CatBoostClassifier
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

with open("assets/SubCategoryClassification/category_0/CatBoosting/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer_catboost = pickle.load(f)

# tfidf_vectorizer_catboost = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer_catboost.transform(X_train)
X_test_tfidf = tfidf_vectorizer_catboost.transform(X_test)
X_oot_tfidf = tfidf_vectorizer_catboost.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer_catboost.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_0/CatBoosting/catboost_model.cbm", "rb") as f:
    model = pickle.load(f)

# model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
# model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall', 'oot']
overall.T

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


,train,test,train_overall,oot
Overall Accuracy,0.713891,0.705828,0.711875,0.704568
Overall F1 Score,0.693318,0.685332,0.691332,0.684194
Overall Recall,0.713891,0.705828,0.711875,0.704568
Overall Precision,0.706695,0.698350,0.704505,0.699756


In [ ]:
# import os
# import joblib
# import numpy as np

# # Base directory path
# base_path = "/assets/"
# model_dir = os.path.join(base_path, "SubCategoryClassification/category_0/CatBoosting/")

# # Create the folder if it doesn't exist
# os.makedirs(model_dir, exist_ok=True)

# # Define paths for saving
# model_path = os.path.join(model_dir, "catboost_model.cbm")
# tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer.pkl")

# # Save the trained CatBoost model
# model.save_model(model_path)
# print(f"CatBoost model saved to {model_path}")

# # Save the TF-IDF vectorizer (if applicable)
# joblib.dump(tfidf_vectorizer_catboost, tfidf_vectorizer_path)
# print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


CatBoost model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_0/CatBoosting/catboost_model.cbm
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_0/CatBoosting/tfidf_vectorizer.pkl


#### Gradient Boosting

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_0_train[~category_0_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_0_train.shape)
test1 = category_0_test[~category_0_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_0_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

with open("assets/SubCategoryClassification/category_0/GradientBoosting/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer_gb = pickle.load(f)

tfidf_vectorizer_gb = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer_gb.transform(X_train)
X_test_tfidf = tfidf_vectorizer_gb.transform(X_test)
X_oot_tfidf = tfidf_vectorizer_gb.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer_gb.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_0/GradientBoosting/gradient_boosting_model.pkl", "rb") as f:
    model = pickle.load(f)

# model = GradientBoostingClassifier()
# model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_gb_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_gb_tfidf.index = ['train', 'test', 'train_overall', 'oot']
overall_gb_tfidf.T

(3795, 16) (3795, 16)
(1357, 16) (1357, 16)
(2846,) (949,)
(2846,) (949,)


,train,test,train_overall,oot
Overall Accuracy,0.943781,0.922023,0.938340,0.929993
Overall F1 Score,0.933696,0.905057,0.926713,0.916926
Overall Recall,0.943781,0.922023,0.938340,0.929993
Overall Precision,0.943560,0.909610,0.935767,0.918259


In [ ]:
# import os
# import joblib
# import numpy as np

# # Base directory path
# base_path = "/assets/"
# model_dir = os.path.join(base_path, "SubCategoryClassification/category_0/GradientBoosting/")

# # Create the folder if it doesn't exist
# os.makedirs(model_dir, exist_ok=True)

# # Define paths for saving
# model_path = os.path.join(model_dir, "gradient_boosting_model.pkl")
# tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer.pkl")

# # Save the trained gradientboosting model
# joblib.dump(model, model_path)
# print(f"GradientBoosting model saved to {model_path}")

# joblib.dump(tfidf_vectorizer_gb, tfidf_vectorizer_path)
# print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


GradientBoosting model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_0/GradientBoosting/gradient_boosting_model.pkl
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_0/GradientBoosting/tfidf_vectorizer.pkl


### Model Inference - to load the saved model and test

####Logistic Regression

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Define paths for loading the model and BoW vectorizer
model_path = "/assets/SubCategoryClassification/category_0/Logistic_Regression/logistic_regression_model.pkl"
bow_vectorizer_path = "/assets/SubCategoryClassification/category_0/Logistic_Regression/bow_vectorizer.pkl"

logreg_model = joblib.load(model_path)
bow_vectorizer = joblib.load(bow_vectorizer_path)

oot_data = test1['SRLL_msg_updated']
X_oot_bow = bow_vectorizer.transform(oot_data)

oot_predictions = logreg_model.predict(X_oot_bow)

oot_labels = test1['subcategory_label']
print("OOT Data Predictions:", oot_predictions)
from sklearn.metrics import accuracy_score, classification_report
accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [24 24 24 ... 24 24 24]
Accuracy on OOT data: 0.92
Classification Report for OOT data:
              precision    recall  f1-score   support

           2       0.56      0.33      0.42       114
          24       0.94      0.98      0.96      1243

    accuracy                           0.92      1357
   macro avg       0.75      0.65      0.69      1357
weighted avg       0.91      0.92      0.91      1357



####Catboosting

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
import catboost

# Define paths for loading the model and TF-IDF vectorizer
model_path = "/assets/SubCategoryClassification/category_0/CatBoosting/catboost_model.cbm"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_0/CatBoosting/tfidf_vectorizer.pkl"

# Load the trained CatBoost model and TF-IDF vectorizer
catboost_model = catboost.CatBoostClassifier()
catboost_model.load_model(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

oot_data = test1['SRLL_msg_updated']

X_oot_tfidf = tfidf_vectorizer.transform(oot_data)

oot_predictions = catboost_model.predict(X_oot_tfidf)

oot_labels = test1['subcategory_label']

print("OOT Data Predictions:", oot_predictions)

accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [24 24 24 ... 24 24 24]
Accuracy on OOT data: 0.93
Classification Report for OOT data:
              precision    recall  f1-score   support

           2       0.79      0.27      0.41       114
          24       0.94      0.99      0.96      1243

    accuracy                           0.93      1357
   macro avg       0.87      0.63      0.68      1357
weighted avg       0.93      0.93      0.92      1357



####Gradient Boosting

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

model_path = "/assets/SubCategoryClassification/category_0/GradientBoosting/gradient_boosting_model.pkl"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_0/GradientBoosting/tfidf_vectorizer.pkl"

gb_model = joblib.load(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

oot_data = test1['SRLL_msg_updated']

X_oot_bow = tfidf_vectorizer.transform(oot_data)

oot_predictions = gb_model.predict(X_oot_bow)

oot_labels = test1['subcategory_label']

print("OOT Data Predictions:", oot_predictions)

accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [24 24 24 ... 24 24 24]
Accuracy on OOT data: 0.93
Classification Report for OOT data:
              precision    recall  f1-score   support

           2       0.69      0.30      0.42       114
          24       0.94      0.99      0.96      1243

    accuracy                           0.93      1357
   macro avg       0.82      0.64      0.69      1357
weighted avg       0.92      0.93      0.92      1357



##Category_1 - Financial	Fraud	Crimes	- Subcategories classification

###Running top 3 models for this category from 5 models(Logistic regression,Naive bayes,random forest,Catboosting,Gradient Boosting)



1.  Logistic regression with TF-IDF   
2.  CatBoosting with BOW
3.  Gradient Boosting with TF-IDF



#### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# tfidf
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

with open("assets/SubCategoryClassification/category_1/Logistic_Regression/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer_lr = pickle.load(f)

# tfidf_vectorizer_lr = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer_lr.transform(X_train)
X_test_tfidf = tfidf_vectorizer_lr.transform(X_test)
X_oot_tfidf = tfidf_vectorizer_lr.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer_lr.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_1/Logistic_Regression/logistic_regression_model.pkl", "rb") as f:
    logreg = pickle.load(f)

# logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
# logreg.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(logreg,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(logreg,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(logreg,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(logreg,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']
overall.T


(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)
44078    27
26873     8
14242    13
44375    27
45371    11
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


,train,test,train_overall,oot
Overall Accuracy,0.713891,0.705828,0.711875,0.704568
Overall F1 Score,0.693318,0.685332,0.691332,0.684194
Overall Recall,0.713891,0.705828,0.711875,0.704568
Overall Precision,0.706695,0.698350,0.704505,0.699756


In [ ]:
# import os
# import joblib
# import numpy as np

# # Base directory path
# base_path = "/assets/"
# model_dir = os.path.join(base_path, "SubCategoryClassification/category_1/Logistic_Regression/")

# # Create the folder if it doesn't exist
# os.makedirs(model_dir, exist_ok=True)

# # Define paths for saving
# model_path = os.path.join(model_dir, "logistic_regression_model.pkl")
# tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer.pkl")

# # Save the trained Logistic Regression model
# joblib.dump(logreg, model_path)
# print(f"Logistic Regression model saved to {model_path}")

# # Save the TF-IDF vectorizer
# joblib.dump(tfidf_vectorizer_lr, tfidf_vectorizer_path)
# print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


Logistic Regression model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_1/Logistic_Regression/logistic_regression_model.pkl
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_1/Logistic_Regression/tfidf_vectorizer.pkl


#### CatBoost

In [ ]:
from catboost import CatBoostClassifier

# BOW
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )

y_train_overall = train1['subcategory_label'].copy()
y_oot = test1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_1/CatBoosting/bow_vectorizer.pkl", "rb") as f:
    bow_vectorizer = pickle.load(f)

# bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

with open("assets/SubCategoryClassification/category_1/CatBoosting/catboost_model.cbm", "rb") as f:
    model = pickle.load(f)

# model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
# model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_bow))]
# Concatenate overall metrics
overall_cat_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_cat_bow.index = ['train', 'test', 'train_overall', 'oot']

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
overall_cat_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.684916,0.683637,0.684596,0.690052
Overall F1 Score,0.655807,0.654264,0.655429,0.661341
Overall Recall,0.684916,0.683637,0.684596,0.690052
Overall Precision,0.669373,0.670252,0.669568,0.675357


In [ ]:
# import os
# import joblib
# import numpy as np

# # Base directory path
# base_path = "/assets/"
# model_dir = os.path.join(base_path, "SubCategoryClassification/category_1/CatBoosting/")

# # Create the folder if it doesn't exist
# os.makedirs(model_dir, exist_ok=True)

# # Define paths for saving
# model_path = os.path.join(model_dir, "catboost_model.cbm")
# bow_vectorizer_path = os.path.join(model_dir, "bow_vectorizer.pkl")

# # Save the trained CatBoost model
# model.save_model(model_path)
# print(f"CatBoost model saved to {model_path}")

# # Save the BOW vectorizer (if applicable)
# joblib.dump(bow_vectorizer, bow_vectorizer_path)
# print(f"TF-IDF vectorizer saved to {bow_vectorizer_path}")


CatBoost model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_1/CatBoosting/catboost_model.cbm
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_1/CatBoosting/bow_vectorizer.pkl


#### Gradient Boosting

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_1_train[~category_1_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_1_train.shape)
test1 = category_1_test[~category_1_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_1_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

with open("assets/SubCategoryClassification/category_1/GradientBoosting/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_1/GradientBoosting/gradient_boosting_model.pkl", "rb") as f:
    model = pickle.load(f)

# model = GradientBoostingClassifier()
# model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_gb_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_gb_tfidf.index = ['train', 'test', 'train_overall', 'oot']
overall_gb_tfidf.T

(52092, 16) (52092, 16)
(17361, 16) (17361, 16)
(39069,) (13023,)
(39069,) (13023,)


,train,test,train_overall,oot
Overall Accuracy,0.733855,0.693235,0.723700,0.697771
Overall F1 Score,0.719633,0.672812,0.708244,0.678226
Overall Recall,0.733855,0.693235,0.723700,0.697771
Overall Precision,0.739317,0.685565,0.727395,0.689429


In [ ]:
# import os
# import joblib
# import numpy as np

# # Base directory path
# base_path = "/assets/"
# model_dir = os.path.join(base_path, "SubCategoryClassification/category_1/GradientBoosting/")

# # Create the folder if it doesn't exist
# os.makedirs(model_dir, exist_ok=True)

# # Define paths for saving
# model_path = os.path.join(model_dir, "gradient_boosting_model.pkl")
# tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer.pkl")

# # Save the trained gradientboosting model
# joblib.dump(model, model_path)
# print(f"GradientBoosting model saved to {model_path}")

# joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
# print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


GradientBoosting model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_1/GradientBoosting/gradient_boosting_model.pkl
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_1/GradientBoosting/tfidf_vectorizer.pkl


### Model Inference - to load the saved model and test

####Logistic Regression

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Define paths for loading the model and BoW vectorizer
model_path = "/assets/SubCategoryClassification/category_1/Logistic_Regression/logistic_regression_model.pkl"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_1/Logistic_Regression/tfidf_vectorizer.pkl"

logreg_model = joblib.load(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

oot_data = test1['SRLL_msg_updated']
X_oot_bow = tfidf_vectorizer.transform(oot_data)

oot_predictions = logreg_model.predict(X_oot_bow)

oot_labels = test1['subcategory_label']
print("OOT Data Predictions:", oot_predictions)
from sklearn.metrics import accuracy_score, classification_report
accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [27 27  8 ... 27  8 16]
Accuracy on OOT data: 0.70
Classification Report for OOT data:
              precision    recall  f1-score   support

           1       0.33      0.03      0.06        89
           3       0.68      0.50      0.58       163
           8       0.76      0.74      0.75      3269
           9       0.67      0.01      0.02       214
          11       0.68      0.39      0.50      1282
          13       0.54      0.43      0.47      1791
          16       0.71      0.56      0.63      2678
          19       1.00      0.02      0.04       133
          27       0.71      0.90      0.79      7742

    accuracy                           0.70     17361
   macro avg       0.67      0.40      0.43     17361
weighted avg       0.70      0.70      0.68     17361



####Catboosting

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
import catboost

# Define paths for loading the model and TF-IDF vectorizer
model_path = "/assets/SubCategoryClassification/category_1/CatBoosting/catboost_model.cbm"
bow_vectorizer_path = "/assets/SubCategoryClassification/category_1/CatBoosting/bow_vectorizer.pkl"

# Load the trained CatBoost model and TF-IDF vectorizer
catboost_model = catboost.CatBoostClassifier()
catboost_model.load_model(model_path)
bow_vectorizer = joblib.load(bow_vectorizer_path)

oot_data = test1['SRLL_msg_updated']

X_oot_tfidf = bow_vectorizer.transform(oot_data)

oot_predictions = catboost_model.predict(X_oot_tfidf)

oot_labels = test1['subcategory_label']

print("OOT Data Predictions:", oot_predictions)

accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [[27]
 [27]
 [ 8]
 ...
 [27]
 [27]
 [16]]
Accuracy on OOT data: 0.69
Classification Report for OOT data:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        89
           3       0.72      0.33      0.45       163
           8       0.76      0.72      0.74      3269
           9       0.00      0.00      0.00       214
          11       0.72      0.35      0.47      1282
          13       0.56      0.31      0.40      1791
          16       0.75      0.53      0.62      2678
          19       0.00      0.00      0.00       133
          27       0.67      0.92      0.78      7742

    accuracy                           0.69     17361
   macro avg       0.46      0.35      0.38     17361
weighted avg       0.68      0.69      0.66     17361



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


####Gradient Boosting

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

model_path = "/assets/SubCategoryClassification/category_1/GradientBoosting/gradient_boosting_model.pkl"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_1/GradientBoosting/tfidf_vectorizer.pkl"

gb_model = joblib.load(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

oot_data = test1['SRLL_msg_updated']

X_oot_bow = tfidf_vectorizer.transform(oot_data)

oot_predictions = gb_model.predict(X_oot_bow)

oot_labels = test1['subcategory_label']

print("OOT Data Predictions:", oot_predictions)

accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [27 27  8 ...  1 27 16]
Accuracy on OOT data: 0.70
Classification Report for OOT data:
              precision    recall  f1-score   support

           1       0.17      0.10      0.13        89
           3       0.53      0.48      0.50       163
           8       0.77      0.73      0.75      3269
           9       0.07      0.02      0.04       214
          11       0.69      0.40      0.50      1282
          13       0.56      0.35      0.43      1791
          16       0.78      0.55      0.64      2678
          19       0.14      0.10      0.12       133
          27       0.69      0.91      0.79      7742

    accuracy                           0.70     17361
   macro avg       0.49      0.40      0.43     17361
weighted avg       0.69      0.70      0.68     17361



## Category_2 - Other	Cyber	Crime	- Subcategories classification

### Running top 3 models for this category from 5 models(Logistic regression,Naive bayes,random forest,Catboosting,Gradient Boosting)



1.  Logistic regression with TF-IDF   
2.  Naive Bayes with TF-IDF  
3.  CatBoosting with BOW



#### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import pandas as pd

# TF-IDF
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(y_train.head())

with open("assets/SubCategoryClassification/category_2/Logistic_Regression/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_2/Logistic_Regression/logistic_regression_model.pkl", "rb") as f:
    logreg = pickle.load(f)

# logreg = LogisticRegression(max_iter=1000 ,multi_class='ovr')
# logreg.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(logreg,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(logreg,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(logreg,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(logreg,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']


(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)
24892     0
10485     0
2770      0
14391     0
12554    15
Name: subcategory_label, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: Undefin

In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.610037,0.581604,0.602929,0.575950
Overall F1 Score,0.558218,0.525947,0.550173,0.520929
Overall Recall,0.610037,0.581604,0.602929,0.575950
Overall Precision,0.562284,0.520230,0.552116,0.520418


In [ ]:
import os
import joblib
import numpy as np

# Base directory path
base_path = "/assets/"
model_dir = os.path.join(base_path, "SubCategoryClassification/category_2/Logistic_Regression/")

# Create the folder if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Define paths for saving
model_path = os.path.join(model_dir, "logistic_regression_model.pkl")
tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer.pkl")

# Save the trained Logistic Regression model
joblib.dump(logreg, model_path)
print(f"Logistic Regression model saved to {model_path}")

# Save the TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


Logistic Regression model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_2/Logistic_Regression/logistic_regression_model.pkl
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_2/Logistic_Regression/tfidf_vectorizer.pkl


#### Naive Bayes

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

with open("assets/SubCategoryClassification/category_2/NaiveBayes/tfidf_vectorizer_nb.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_2/NaiveBayes/naive_bayes_model.pkl", "rb") as f:
    nb = pickle.load(f)

# nb = MultinomialNB()
# nb.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(nb,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(nb,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(nb,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(nb,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.583689,0.577765,0.582208,0.563873
Overall F1 Score,0.518286,0.507196,0.515559,0.493893
Overall Recall,0.583689,0.577765,0.582208,0.563873
Overall Precision,0.545862,0.513808,0.539285,0.518851


In [ ]:
import os
import joblib
import numpy as np

# Base directory path
base_path = "/assets/"
model_dir = os.path.join(base_path, "SubCategoryClassification/category_2/NaiveBayes/")

# Create the folder if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Define paths for saving
model_path = os.path.join(model_dir, "naive_bayes_model.pkl")
tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer_nb.pkl")

# Save the trained Naive Bayes model
joblib.dump(nb, model_path)
print(f"Naive Bayes model saved to {model_path}")

# Save the TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


Naive Bayes model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_2/NaiveBayes/naive_bayes_model.pkl
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_2/NaiveBayes/tfidf_vectorizer_nb.pkl


#### CatBoost

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = category_2_train[~category_2_train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, category_2_train.shape)
test1 = category_2_test[~category_2_test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, category_2_test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['subcategory_label'], test_size=0.25, random_state=42, stratify=train1['subcategory_label']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

with open("assets/SubCategoryClassification/category_2/CatBoosting/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['subcategory_label'].copy()
y_train_overall = train1['subcategory_label'].copy()

with open("assets/SubCategoryClassification/category_2/CatBoosting/catboost_model.cbm", "rb") as f:
    tfidf_vectorizer = pickle.load(f)
# model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
# model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])

# Adjust the index dynamically based on the row count
classwise.index = [f"{set_name}_{i}" for set_name in ["train", "test", "train_overall", "oot"] for i in range(len(train_class_wise_tfidf))]
# Concatenate overall metrics
overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall', 'oot']

(28136, 16) (28136, 16)
(9605, 16) (9605, 16)
(21102,) (7034,)
(21102,) (7034,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [ ]:
overall_cat_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.574732,0.564828,0.572256,0.562936
Overall F1 Score,0.516291,0.506449,0.513834,0.503782
Overall Recall,0.574732,0.564828,0.572256,0.562936
Overall Precision,0.519055,0.501548,0.514544,0.506421


In [ ]:
import os
import joblib
import numpy as np

# Base directory path
base_path = "/assets/"
model_dir = os.path.join(base_path, "SubCategoryClassification/category_2/CatBoosting/")

# Create the folder if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Define paths for saving
model_path = os.path.join(model_dir, "catboost_model.cbm")
tfidf_vectorizer_path = os.path.join(model_dir, "tfidf_vectorizer.pkl")

# Save the trained CatBoost model
model.save_model(model_path)
print(f"CatBoost model saved to {model_path}")

# Save the TF-IDF vectorizer (if applicable)
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
print(f"TF-IDF vectorizer saved to {tfidf_vectorizer_path}")


CatBoost model saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_2/CatBoosting/catboost_model.cbm
TF-IDF vectorizer saved to /content/drive/MyDrive/IndiaAI_Hackathon/assets/SubCategoryClassification/category_2/CatBoosting/tfidf_vectorizer.pkl


### Model Inference - to load the saved model and test

####Logistic Regression

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Define paths for loading the model and BoW vectorizer
model_path = "/assets/SubCategoryClassification/category_2/Logistic_Regression/logistic_regression_model.pkl"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_2/Logistic_Regression/tfidf_vectorizer.pkl"

logreg_model = joblib.load(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

oot_data = test1['SRLL_msg_updated']
X_oot_bow = tfidf_vectorizer.transform(oot_data)

oot_predictions = logreg_model.predict(X_oot_bow)

oot_labels = test1['subcategory_label']
print("OOT Data Predictions:", oot_predictions)
from sklearn.metrics import accuracy_score, classification_report
accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [17  0  0 ...  0  0  0]
Accuracy on OOT data: 0.58
Classification Report for OOT data:
              precision    recall  f1-score   support

           0       0.61      0.86      0.71      3599
           4       0.64      0.69      0.66      1328
           5       0.00      0.00      0.00        51
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00       171
          10       0.00      0.00      0.00       187
          12       0.61      0.29      0.39       208
          14       0.00      0.00      0.00       239
          15       0.58      0.35      0.44      1444
          17       0.41      0.97      0.58       541
          18       0.00      0.00      0.00        61
          20       0.47      0.21      0.29       290
          21       0.00      0.00      0.00        37
          22       0.58      0.40      0.47       735
          23       0.70      0.06      0.10       124
          26       0.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


####Catboosting

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
import catboost

# Define paths for loading the model and TF-IDF vectorizer
model_path = "/assets/SubCategoryClassification/category_2/CatBoosting/catboost_model.cbm"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_2/CatBoosting/tfidf_vectorizer.pkl"

# Load the trained CatBoost model and TF-IDF vectorizer
catboost_model = catboost.CatBoostClassifier()
catboost_model.load_model(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)

oot_data = test1['SRLL_msg_updated']

X_oot_tfidf = tfidf_vectorizer.transform(oot_data)

oot_predictions = catboost_model.predict(X_oot_tfidf)

oot_labels = test1['subcategory_label']

print("OOT Data Predictions:", oot_predictions)

accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")

report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [[17]
 [ 0]
 [ 0]
 ...
 [15]
 [ 0]
 [ 0]]
Accuracy on OOT data: 0.56
Classification Report for OOT data:
              precision    recall  f1-score   support

           0       0.58      0.88      0.70      3599
           4       0.67      0.58      0.62      1328
           5       0.00      0.00      0.00        51
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00       171
          10       0.00      0.00      0.00       187
          12       0.77      0.24      0.36       208
          14       0.00      0.00      0.00       239
          15       0.57      0.32      0.42      1444
          17       0.41      0.97      0.58       541
          18       0.00      0.00      0.00        61
          20       0.47      0.19      0.27       290
          21       0.00      0.00      0.00        37
          22       0.57      0.41      0.47       735
          23       0.00      0.00      0.00       124
        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


####Naive Bayes

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB

model_path = "/assets/SubCategoryClassification/category_2/NaiveBayes/naive_bayes_model.pkl"
tfidf_vectorizer_path = "/assets/SubCategoryClassification/category_2/NaiveBayes/tfidf_vectorizer_nb.pkl"
nb_model = joblib.load(model_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
oot_data = test1['SRLL_msg_updated']  # Modify based on your OOT data
X_oot_tfidf = tfidf_vectorizer.transform(oot_data)
oot_predictions = nb_model.predict(X_oot_tfidf)
oot_labels = test1['subcategory_label']  # Replace with your actual labels for OOT data
print("OOT Data Predictions:", oot_predictions)

accuracy_oot = accuracy_score(oot_labels, oot_predictions)
print(f"Accuracy on OOT data: {accuracy_oot:.2f}")
report_oot = classification_report(oot_labels, oot_predictions)
print("Classification Report for OOT data:")
print(report_oot)


OOT Data Predictions: [17  0  0 ...  0  0  0]
Accuracy on OOT data: 0.56
Classification Report for OOT data:
              precision    recall  f1-score   support

           0       0.59      0.88      0.71      3599
           4       0.60      0.73      0.66      1328
           5       0.00      0.00      0.00        51
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00       171
          10       0.00      0.00      0.00       187
          12       0.78      0.07      0.12       208
          14       0.00      0.00      0.00       239
          15       0.61      0.27      0.37      1444
          17       0.41      0.97      0.58       541
          18       0.00      0.00      0.00        61
          20       0.42      0.08      0.14       290
          21       0.00      0.00      0.00        37
          22       0.56      0.36      0.44       735
          23       1.00      0.03      0.06       124
          26       0.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
